<a href="https://colab.research.google.com/github/AiJared/DCGAN/blob/main/DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import keras
from IPython import display

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class DCGAN:
  def __init__(self, image_shape=(64, 64, 1), latent_dim=100):
    """
    Initialize DCGAN with specific image dimensions and latent space

    Args:
      image_shape(tuple): the dimensions of input images
      latent_dim(int): Dimensionality of random noice vector
    """
    self.image_shape = image_shape
    self.latent_dim = latent_dim

    # Key DCGAN improvements: Specific architectural guidelines
    self.generator = self.build_generator()
    self.discriminator = self.build_discriminator()

    self.adversarial_model = self.build_adversarial_model()

    def build_generator(self):
      """
      Build DCGAN generator with convolutional transpose layers

      Key DCGAN features:
       - Transposed convolutional layers
       - Batch normalization
       - LeakyReLu activations
      """

      model = keras.Sequential([
          # input layer: Dense to initial feature map
          keras.layers.Dense(8 * 8 * 256, input_dim=self.latent_dim),
          keras.layes.Reshape(8, 8, 256),

          # upsampling blocks with batch norm and leakyReLu
          keras.layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding='same'),
          keras.layers.BatchNormalization(),
          keras.layers.LeakyReLU(alpha=0.2),

          # Final layer to generate image

          keras.layers.Conv2DTranspose(self.image_shape[2], kernel_size=4, strides=2,
                                       padding='same', activation='tanh')
      ])

      return model

    def build_discriminator(self):
      """
      Build DCGAN discriminator using convolutional layers

      Key DCGAN features:
       - Strided convolutional layers
       - Batch normalization
       - LeakyReLu activations
       - No fully connected layers
      """
      model = keras.Sequential([
          # convolution layers with strided convolutions
          keras.layers.Conv2D(64, kernel_size=4, strides=2, padding='same',
                              input_shape=self.image_shape),
          keras.layers.LeakyReLu(alpha=0.2),
          keras.layers.Conv2D(128, kernel_size=4, strides=2, padding='same'),
          keras.layers.BatchNormalization(),
          keras.layers.LeakyReLu(alpha=0.2),

          # Flatten and output
          keras.layers.Flatten(),
          keras.layers.Dense(1, activation='sigmoid')
      ])

      return model

    def build_adversarial_model(self):
      """
      Build combined generator-discriminator model.

      Ensures discriminator model is non-trainable during generator training
      """
      # Freeze discriminator weights during generator training
      self.discriminator.trainable = False

      model = keras.Sequential([
          self.generator,
          self.discriminator
      ])

      return model

    def train_step(self, real_images):
      """
      Single training step for DCGAN

      Implements adversarial training with gradient computation
      """
      batch_size = tf.shape(real_images)[0]

      # Generate random noice
      noise = tf.random.normal([batch_size, self.latent_dim])

      # Generate fake images
      generated_images = self.generator(noise, training=True)

      # prepare labels
      real_labels = tf.ones((batch_size, 1))
      fake_labels = tf.zeros((batch_size, 1))

      # Train Discriminator
      with tf.GradientTape as disc_tape:
        real_predictions = self.discriminator(real_images, training=True)
        fake_predictions = self.discriminator(generated_images, training=True)

        disc_real_loss = keras.losses.binary_crossentropy(real_labels, real_predictions)
        disc_fake_loss = keras.losses.binary_crossentropy(fake_labels, fake_predictions)

        disc_loss = 0.5 * (disc_real_loss + disc_fake_loss)

      # Compute and appy discriminator gradients
      disc_gradients = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)
      keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5).apply_gradients(
          zip(disc_gradients, self.discriminator.trainable_variables)
      )

      # Train Generator
      with tf.GradientTape as gen_tape:
        noise = tf.random.normal([batch_size, self.latent_dim])
        generated_images = self.generator(noise, training=True)

        fake_predictions = self.discriminator(generated_images, training=True)
        gen_loss = keras.losses.binary_crossentropy(real_labels, fake_predictions)

      # Compute and apply generated gradients
      gen_gradients = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
      keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5).apply_gradients(
          zip(gen_gradients, self.generator.trainable_variables)
      )

      return disc_loss, gen_loss

    def train(self, dataset, epochs=100):
      """
      Train the GAN

      Args:
       dataset (tf.data.Dataset): Training dataset
       epochs (int): Number of training epochs
      """
      for epoch in range(epochs):
        for batch in dataset:
          disc_loss, gen_loss = self.train_step(batch)

        # Generate and save images periodically
        if epoch % 10 == 0:
          self.generate_and_save_images(epoch)
          print(f"Epoch {epoch}: Discriminator Loss: {disc_loss.numpy()}, Generator Loss: {gen_loss.numpy()}")

    def generate_and_save_images(self, epoch, num_examples=16):
      """
      Generate sample images during training

      Args:
        epoch (int): Current training epoch
        num_examples (int): Number of images to generate
      """